In [238]:
import pandas as pd
import numpy as np

In [239]:
df = pd.read_csv('ABC.txt', sep=',', header=0, parse_dates=['date_t'], 
                 date_parser=lambda d: pd.datetime.strptime(d, "%Y/%m/%d %H") if d!='date_t' else pd.NaT,
                 dtype={'msisdn':str, 'imsi':str}) 
df.drop(df[df['date_t'].isnull()].index, inplace=True)

In [240]:
grouped_df = df.groupby(['msisdn', 'imsi', df['date_t'].dt.date]).size().reset_index(name='hours')

In [254]:
def consecutive_days(arr):
    return (arr.iloc[-1] - arr.iloc[0]).days + 1

def merge_consecutive_date(df):
    sorted_date = df.sort_values(by=['date_t'])[['date_t', 'hours']]

    consecutived = sorted_date['date_t']-pd.to_timedelta(1, unit='D') != sorted_date['date_t'].shift(1)
    #if len(consecutived) > 1:
    #    consecutived.iloc[0] = consecutived.iloc[1]
    consecutived = consecutived.astype(np.int32)

    merged_date = sorted_date.groupby(consecutived.cumsum(),as_index=False).agg({
                                 'date_t':[('start_date', 'first'), ('days', consecutive_days)], 
                                 'hours':[('sum_hours','sum')]})
    merged_date.columns = merged_date.columns.droplevel(0)

    return merged_date

In [1]:
merged_df = grouped_df.groupby(['msisdn', 'imsi']).apply(merge_consecutive_date)
merged_df = merged_df.reset_index(level=2, drop=True)

NameError: name 'grouped_df' is not defined

In [256]:
merged_df.to_csv(r'OB_UK_merged.csv', sep=',', header=True, index=True, na_rep='',
                 columns=['start_date', 'days', 'sum_hours'])